In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
from math import atan2, pi, sqrt, cos, sin, floor
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2)  
from tensorflow.keras.layers import Dense, MaxPooling1D, Flatten, LSTM, Bidirectional, concatenate
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.compat.v1.keras.backend as K
from tensorflow.keras.models import load_model
from tcn import TCN, tcn_full_summary
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform
from keras_flops import get_flops
import pickle
import csv
import random
import itertools
import math
import time
sys.path.insert(0, '../dataset2/')
from data_utils_2 import *
from vetorch_traj_utils import *

## Import Dataset

In [ ]:
window_size = 100
stride = 20

f = '../dataset2/'
X_train,Y_Pos_train, Physics_Vec_train, x_vel_train, y_vel_train, x0_list_train, y0_list_train, size_of_each_train = import_agrobot_dataset_p3(dataset_folder=f, type_flag=1, window_size=window_size, stride=stride)
P = np.repeat(Physics_Vec_train,window_size).reshape((Physics_Vec_train.shape[0],window_size,1))
X_train = np.concatenate((X_train,P),axis=2)

X_test,Y_Pos_test, Physics_Vec_test, x_vel_test, y_vel_test, x0_list_test, y0_list_test, size_of_each_test= import_agrobot_dataset_p3(type_flag = 2, dataset_folder=f,window_size=window_size, stride=stride)
P_test = np.repeat(Physics_Vec_test,window_size).reshape((Physics_Vec_test.shape[0],window_size,1))
X_test = np.concatenate((X_test,P_test),axis=2)

## Generate Heading, Displacement and Timestamp Intervals

In [ ]:
disp_train = np.sqrt((x_vel_train**2) + (y_vel_train**2)) 
disp_test = np.sqrt((x_vel_test**2) + (y_vel_test**2)) 

In [ ]:
heading_train = np.zeros((Y_Pos_train.shape[0]))
prev = 0
for i in range(Y_Pos_train.shape[0]):
    theta = abs_heading(Y_Pos_train[i,-1,0],Y_Pos_train[i,-1,1],Y_Pos_train[i,0,0],Y_Pos_train[i,0,1])
    if theta<180:
        theta = theta + 180

    heading_train[i] = theta - prev
    if(heading_train[i]>100 or heading_train[i]<-100):
        theta2 = theta
        prev2 = prev
        if theta<prev:
            theta2 = theta + 360
        else:
            prev2 =  prev + 360
        heading_train[i] = theta2 - prev2 
    prev = theta

In [ ]:
heading_test = np.zeros((Y_Pos_test.shape[0]))
prev = 0
for i in range(Y_Pos_test.shape[0]):
    theta = abs_heading(Y_Pos_test[i,-1,0],Y_Pos_test[i,-1,1],Y_Pos_test[i,0,0],Y_Pos_test[i,0,1])
    if theta<180:
        theta = theta + 180

    heading_test[i] = theta - prev
    if(heading_test[i]>100 or heading_test[i]<-100):
        theta2 = theta
        prev2 = prev
        if theta<prev:
            theta2 = theta + 360
        else:
            prev2 =  prev + 360
        heading_test[i] = theta2 - prev2 
    prev = theta

In [ ]:
ts_list_train = 0.01*np.ones((X_train.shape[0],window_size,1))
ts_list_test = 0.01*np.ones((X_test.shape[0],window_size,1))

## Model Training

In [ ]:
input_acc = Input(shape=(window_size
                         , 3))
r = TCN(kernel_size=7,nb_filters=64,nb_stacks=4,use_skip_connections=True,use_weight_norm=True,
        dropout_rate=0.2,dilations=(1,2,4,8),return_sequences=True)(input_acc)
slice_layer = r[:,-1,:]
fc = Dense(64)(slice_layer)
fc = Dense(32)(fc)
output1 = Dense(1, activation='linear', name='disp')(fc)
model = Model(inputs=[input_acc],outputs=[output1])
model.summary()

input_gyro = Input(shape=(window_size, 3))
s = TCN(kernel_size=7,nb_filters=64,nb_stacks=4,use_skip_connections=True,use_weight_norm=True,
        dropout_rate=0.2,dilations=(1,2,4,8),return_sequences=True)(input_gyro)
slice_layer_2 = s[:,-1,:]
fc_2 = Dense(64)(slice_layer_2)
fc_2 = Dense(32)(fc_2)
output2 = Dense(1, activation='linear', name='ori')(fc_2)
model2 = Model(inputs=[input_gyro],outputs=[output2])
model2.summary()

In [ ]:
model_name = 'vetorch_dset_2_disp.hdf5'
opt = tf.keras.optimizers.Adam(learning_rate=0.0015)
model.compile(loss={'disp': 'mse'},optimizer=opt)
checkpoint = ModelCheckpoint(model_name, monitor='loss', verbose=1, save_best_only=True)
model.fit(x=X_train[:,:,0:3],y=[disp_train],
          callbacks=[checkpoint],epochs=1500,batch_size=32,verbose=1,shuffle=True)

In [ ]:
model2_name = 'vetorch_dset_2_head.hdf5'
opt2 = tf.keras.optimizers.Adam(learning_rate=0.0015)
model2.compile(loss={'ori': 'mse'},optimizer=opt2)
checkpoint2 = ModelCheckpoint(model2_name, monitor='loss', verbose=1, save_best_only=True)
model2.fit(x=X_train[:,:,3:6],y=[heading_train],
          callbacks=[checkpoint2],epochs=1500,batch_size=32,verbose=1,shuffle=True)

## Evaluation

In [ ]:
model_name = 'vetorch_dset_2_disp.hdf5'
model2_name = 'vetorch_dset_2_head.hdf5'
model = load_model(model_name,custom_objects={'TCN':TCN})
model2 = load_model(model2_name,custom_objects={'TCN':TCN})

In [ ]:
train1_mat = X_train[:,:,0:3]
train2_mat = X_train[:,:,3:6]
test1_mat = X_test[:,:,0:3]
test2_mat = X_test[:,:,3:6]

Unseen Trajectory

In [ ]:
ATE = []
RTE = []
ATE_dist = []
RTE_dist = []
for i in range(len(size_of_each_test)):
    Pvx, Pvy = vetorch_pos_generator(test1_mat, test2_mat, size_of_each_test, 
                   x0_list_test, y0_list_test, window_size, stride,i,model,model2)   
    Gvx, Gvy = vetorch_GT_pos_generator(disp_test,heading_test,size_of_each_test,
                                x0_list_test, y0_list_test, window_size, stride,i)
    
    at, rt, at_all, rt_all = Cal_TE(Gvx, Gvy, Pvx, Pvy,
                                    sampling_rate=100,window_size=window_size,stride=stride)
    ATE.append(at)
    RTE.append(rt)
    ATE_dist.append(Cal_len_meters(Gvx, Gvy))
    RTE_dist.append(Cal_len_meters(Gvx, Gvy, 600))
    print('ATE, RTE, Trajectory Length, Trajectory Length (60 seconds)',ATE[i],RTE[i],ATE_dist[i],RTE_dist[i])
    
print('Median ATE and RTE', np.median(ATE),np.median(RTE))
print('Mean ATE and RTE', np.mean(ATE),np.mean(RTE))
print('STD ATE and RTE', np.std(ATE),np.std(RTE))

Seen Trajectory

In [ ]:
ATE = []
RTE = []
ATE_dist = []
RTE_dist = []
for i in range(len(size_of_each_train)):
    Pvx, Pvy = vetorch_pos_generator(train1_mat, train2_mat, size_of_each_train, 
                   x0_list_train, y0_list_train, window_size, stride,i,model,model2)   
    Gvx, Gvy = vetorch_GT_pos_generator(disp_train,heading_train,size_of_each_train,
                                x0_list_train, y0_list_train, window_size, stride,i)
    
    at, rt, at_all, rt_all = Cal_TE(Gvx, Gvy, Pvx, Pvy,
                                    sampling_rate=100,window_size=window_size,stride=stride)
    ATE.append(at)
    RTE.append(rt)
    ATE_dist.append(Cal_len_meters(Gvx, Gvy))
    RTE_dist.append(Cal_len_meters(Gvx, Gvy, 600))
    print('ATE, RTE, Trajectory Length, Trajectory Length (60 seconds)',ATE[i],RTE[i],ATE_dist[i],RTE_dist[i])
    
print('Median ATE and RTE', np.median(ATE),np.median(RTE))
print('Mean ATE and RTE', np.mean(ATE),np.mean(RTE))
print('STD ATE and RTE', np.std(ATE),np.std(RTE))